# Training an MT5 model for Czech paraphrasing

Main configuration

In [1]:
initial_finetuning = True  # this is true only at the beginning of fine-tuning. Set to False if you want to continue training from some checkpoint saved on google drive.
hf_checkpoint = 'google/mt5-small'
drive_checkpoint = ''  # e.g. '/content/drive/MyDrive/models/old-checkpoint-234/'

## Environment Setup

We need a GPU, so we check the availability:

In [2]:
!nvidia-smi

Mon May 22 18:41:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

And we install all neede libraries.

In [4]:
!pip install datasets==2.11.0 transformers==4.28.0 nltk==3.8.1 parascore==1.0.5 sentencepiece==0.1.98

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


We store checkpoints on Google Drive. After we have mounted our Google Drive, the root folder of our Drive is at `/content/drive/MyDrive/`.

In [6]:
from google.colab import drive
drive.mount("/content/drive/")

MessageError: ignored

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Download and Preparation

In [8]:
from datasets import load_dataset

We use our own created dataset for german-german paraphrases.

In [9]:
raw_dataset = load_dataset('yawnick/para_crawl_cscs')
raw_dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/yawnick___csv/yawnick--para_crawl_cscs-8346f3eb1e22ef37/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Original', 'Paraphrase'],
        num_rows: 55544
    })
    test: Dataset({
        features: ['Original', 'Paraphrase'],
        num_rows: 11532
    })
    validation: Dataset({
        features: ['Original', 'Paraphrase'],
        num_rows: 9803
    })
})

Let's store the splits separately and look at one example.

In [10]:
raw_dataset_train = raw_dataset['train']
raw_dataset_val = raw_dataset['validation']
raw_dataset_test = raw_dataset['test']
raw_dataset_train[5]

{'Original': 'A určitě po odehrání jednou, budete chtít vrátit a pokračovat ve hře v čase.',
 'Paraphrase': 'A určitě po hraní jednou, budete chtít vrátit a pokračovat ve hře v čase.'}

Now, let's prepare the data for training.

In [11]:
from transformers import T5Tokenizer, AutoTokenizer

In [12]:
tokenizer = AutoTokenizer.from_pretrained(hf_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Let's see how the tokenizer works:

In [13]:
s1 = raw_dataset_train[4]['Original']
s2 = raw_dataset_train[4]['Paraphrase']
print(s1)
print(s2)
inputs = tokenizer(s1, text_target=s2)
print([tokenizer.decode(id) for id in inputs['input_ids']])
inputs

Plachtění je také nazýván, jít prozkoumat.
Plachtění se také nazývá, jít prozkoumat.
['P', 'lacht', 'ění', 'je', 'tak', 'é', 'naz', 'ý', 'ván', ',', '', 'jít', 'pro', 'zkou', 'mat', '.', '</s>']


{'input_ids': [363, 118696, 21941, 384, 756, 361, 36940, 952, 55397, 261, 259, 49042, 731, 90602, 4269, 260, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [363, 118696, 21941, 303, 756, 361, 36940, 952, 12499, 261, 259, 49042, 731, 90602, 4269, 260, 1]}

Now we create a preprocess function that turns a dataset item into a form that the model can use for training.

In [14]:
max_length = 128

# the prefix has to (dynamically) be adjusted depending on the language or when training multilingually (I think).
prefix = 'paraphrase: '

def preprocess_function(examples):
    inputs = [prefix+s1 for s1 in examples['Original']]
    targets = examples['Paraphrase']
    # most likely there will be nothing to truncate, but we still add it
    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_length, truncation=True)
    return model_inputs

Now we apply the preprocessing function to the datasets.

In [15]:
tokenized_ds_train = raw_dataset_train.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_dataset_train.column_names
)
tokenized_ds_val = raw_dataset_val.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_dataset_val.column_names
)
tokenized_ds_test = raw_dataset_test.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_dataset_test.column_names
)

Map:   0%|          | 0/55544 [00:00<?, ? examples/s]

Map:   0%|          | 0/9803 [00:00<?, ? examples/s]

Map:   0%|          | 0/11532 [00:00<?, ? examples/s]

Now the data is ready.

## Model and Training Preparation

Next, the model and a Datacollator.

In [16]:
from transformers import MT5ForConditionalGeneration

Either load the pretrained model from huggingface at the beginning of fine-tuning for the first epoch, or load the model from a previous fine-tune checkooint from google drive.

In [17]:
if initial_finetuning:
  model = MT5ForConditionalGeneration.from_pretrained(hf_checkpoint)
else:
  model = MT5ForConditionalGeneration.from_pretrained(drive_checkpoint)

Next, we instantiate a DataCollator.

In [18]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Here, I'll skip the example usage of the datacollator, check it out [here](https://huggingface.co/learn/nlp-course/chapter7/4?fw=pt#data-collation).

Now, let's continue with metrics. We will use Parascore.

In [19]:
from parascore import ParaScorer

scorer = ParaScorer(lang='cs')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Let's quickly go over how Parascore is used (this example is in english, so it's not ideal):

In [20]:
cands = ["A young person is skating.", "I like sports.", "He catches the ball.", "That's very interesting!"]
sources = ["There's a child on a skateboard.", "I like to relax.", "good morning, everyone!", "I find this interesting."]
score = scorer.free_score(cands, sources)
float(score[-1].mean())

0.7657963037490845

Now, here's the `compute_metrics` function (mostly copied from [here](https://huggingface.co/learn/nlp-course/chapter7/4?fw=pt#metrics)):

In [21]:
import numpy as np

In [22]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]
    print(decoded_preds[:5])
    print(decoded_labels[:5])
    
    parascore = scorer.free_score(decoded_preds, decoded_labels)
    return {'parascore': float(parascore[-1].mean())}
    

In [23]:
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer

In [25]:


args = Seq2SeqTrainingArguments(
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy='steps',
    logging_steps=500,
    output_dir='/content/drive/MyDrive/models/mono-cscs',  # this is where the checkpoint will be saved
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
)

In [26]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_ds_train,
    eval_dataset=tokenized_ds_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [27]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Parascore
1,2.476900,1.715271,0.846561
2,2.158100,1.592880,0.850268
3,2.034000,1.539779,0.850638
4,1.978300,1.509774,0.850826
5,1.925800,1.504252,0.850907


['Potřebat různé úrovně péče lze očekávat od každ', 'Úloha příležitost představuje příležitost říci lidem, co můžete nabí', 'Každý návštěvník se ptal na účel pobytu, ubytování', 'Vytvořte krásnou krajinu, terénní vlastnosti, zahrady a', 'Bezpečně zaplatil Hoy službu za nízkou cenu!']
['Pochopte různou úroveň péče, kterou můžete očekávat od každého zařízení.', 'Pracovní příležitost je příležitost říct lidem, co můžete nabídnout, stejně jako to, co potřebujete.', 'Každý návštěvník se bude ptát na účel pobytu, ubytování a případného zaměstnání.', 'Vytvořte si krásnou krajinu, terénní vlastnosti, zahrady a paluby pro ideální venkovní obývací prostor!', 'Bezpečné placené hoy služby za nízkou cenu!']
['Pochopit různé úrovně péče lze očekávat od každ', 'Úloha příležitost představuje příležitost říci lidem, co můžete nabí', 'Každý návštěvník se ptal na účel pobytu, ubytování', 'Vytvořte krásnou krajinu, terénní vlastnosti, zahrady a', 'Bezpečně zaplatil Hoy službu za nízkou cenu!']
['Pochopte 

TrainOutput(global_step=17360, training_loss=2.467701668453656, metrics={'train_runtime': 8662.3182, 'train_samples_per_second': 32.061, 'train_steps_per_second': 2.004, 'total_flos': 1.516077379289088e+16, 'train_loss': 2.467701668453656, 'epoch': 5.0})